In [ ]:
MOMCS - ZDT 1, 2 and 3

In [ ]:
import numpy as np
import os
import sys
import random
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.special import gamma


In [ ]:
def zdt1(x):
    """Evaluate ZDT1 objectives."""
    n = len(x)
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / (n - 1)
    f2 = g * (1 - np.sqrt(f1 / g))
    return [f1, f2]


In [ ]:
def zdt2(x):
    """Evaluate ZDT2 objectives."""
    n = len(x)
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / (n - 1)
    f2 = g * (1 - (f1 / g) ** 2)
    return [f1, f2]


In [ ]:
def zdt3(x):
    """Evaluate ZDT3 objectives."""
    n = len(x)
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / (n - 1)
    f2 = g * (1 - np.sqrt(f1 / g) - (f1 / g) * np.sin(10 * np.pi * f1))
    return [f1, f2]


In [ ]:
def dominates_pair(obj_a, obj_b):
    """Returns True if obj_a dominates obj_b."""
    return (obj_a[0] < obj_b[0] and obj_a[1] <= obj_b[1]) or \
           (obj_a[0] <= obj_b[0] and obj_a[1] < obj_b[1])


In [ ]:
def initialize_population(num_nests, bounds):
    """Initialize nests with random values."""
    return np.random.uniform(low=bounds[:, 0], high=bounds[:, 1], size=(num_nests, bounds.shape[0]))

def evaluate_population(population):
    """Evaluate fitness values for a population."""
    return np.array([zdt1(individual) for individual in population])


In [ ]:
def calculate_crowding_distance(front):
    """Calculate crowding distance for each individual in a Pareto front."""
    if len(front) == 0:
        return []
    
    num_objectives = len(front[0])
    distances = np.zeros(len(front))
    front_size = len(front)
    
    # Convert the front to a numpy array for easier manipulation
    front_array = np.array(front)
    
    for i in range(num_objectives):
        # Sort indices based on the current objective
        sorted_indices = np.argsort(front_array[:, i])
        distances[sorted_indices[0]] = distances[sorted_indices[-1]] = np.inf
        min_obj = front_array[sorted_indices[0], i]
        max_obj = front_array[sorted_indices[-1], i]
        
        if max_obj == min_obj:
            # If all values are the same for this objective, skip distance calculation
            continue
        
        for j in range(1, front_size - 1):
            distances[sorted_indices[j]] += (front_array[sorted_indices[j + 1], i] - front_array[sorted_indices[j - 1], i]) / (max_obj - min_obj)
    
    return distances


In [ ]:
def levy_flight(size, beta=1.5):
    """Generate Lévy flight step."""
    sigma_u = (gamma(1 + beta) * np.sin(np.pi * beta / 2) / (gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
    u = np.random.normal(0, sigma_u, size=size)
    v = np.random.normal(0, 1, size=size)
    step = u / (np.abs(v) ** (1 / beta))
    return step


In [ ]:
def efficient_non_dominated_sort(objective_values):
    num_individuals = len(objective_values)
    dominated_count = np.zeros(num_individuals, dtype=int)
    dominates = [set() for _ in range(num_individuals)]
    
    for i in range(num_individuals):
        for j in range(i + 1, num_individuals):
            if dominates_pair(objective_values[i], objective_values[j]):
                dominates[i].add(j)
                dominated_count[j] += 1
            elif dominates_pair(objective_values[j], objective_values[i]):
                dominates[j].add(i)
                dominated_count[i] += 1
    
    fronts = []
    current_front = np.where(dominated_count == 0)[0].tolist()
    
    while current_front:
        fronts.append(current_front)
        next_front = []
        for i in current_front:
            for j in dominates[i]:
                dominated_count[j] -= 1
                if dominated_count[j] == 0:
                    next_front.append(j)
        current_front = next_front
    
    return fronts


In [ ]:
def run_momcs(num_nests, max_evaluations, A, phi, bounds):
    num_objectives = 2  # For ZDT1
    num_dimensions = bounds.shape[0]
    population = initialize_population(num_nests, bounds)
    objective_values = evaluate_population(population)
    num_evaluations = num_nests
    generation = 1
    
    pareto_optimal = []
    pareto_front = []
    optimisation_archive = []
    
    while num_evaluations < max_evaluations:
        # Non-dominated sorting and crowding distance calculation
        fronts = efficient_non_dominated_sort(objective_values)
        
        pareto_optimal.append((population[i], objective_values[i]) for i in fronts[0])
        pareto_front.extend(np.array(objective_values[i]) for i in fronts[0])
        
        sorted_nests = []
        sorted_objectives = []
        
        for front in fronts:
            front_nests = [population[i] for i in front]
            front_objectives = [objective_values[i] for i in front]
            distances = calculate_crowding_distance(front_objectives)
            
            # Sort the front by crowding distance in descending order
            sorted_indices = np.argsort(distances)[::-1]
            sorted_nests.extend(np.array(front_nests)[sorted_indices])
            sorted_objectives.extend(np.array(front_objectives)[sorted_indices])
        
        optimisation_archive.append((sorted_nests, sorted_objectives))
        
        # Abandon bottom 20% of nests
        n_abandon = int(0.2 * num_nests)
        for i in range(n_abandon):
            xi = sorted_nests[-(i + 1)]  # Select the nest to be abandoned
            alpha = A / np.cbrt(generation)  # Lévy flight step size
            step = levy_flight(num_dimensions)
            xk = xi + alpha * step  # New egg via Lévy flight
            xk = np.clip(xk, bounds[:, 0], bounds[:, 1])
            obj_values = zdt1(xk)
            
            sorted_nests[-(i + 1)] = xk
            sorted_objectives[-(i + 1)] = obj_values
            num_evaluations += 1
        
        # Refinement of top nests
        n_top = num_nests - n_abandon
        for i in range(n_top):
            xi = sorted_nests[i]
            xj_index = np.random.choice(n_top)
            xj = sorted_nests[xj_index]
            
            if np.array_equal(xi, xj):
                alpha = A / (generation**2)  # Lévy flight step size for same nest
                step = levy_flight(num_dimensions)
                xk = xi + alpha * step
                xk = np.clip(xk, bounds[:, 0], bounds[:, 1])
            else:
                dx = np.abs(np.array(xi) - np.array(xj)) / phi
                objective_xi = sorted_objectives[i]
                objective_xj = sorted_objectives[xj_index]
                
                if dominates_pair(objective_xi, objective_xj):
                    xk = np.array(xj) + dx * np.sign(np.array(xi) - np.array(xj))
                elif dominates_pair(objective_xj, objective_xi):
                    xk = np.array(xi) + dx * np.sign(np.array(xj) - np.array(xi))
                else:
                    alpha = A / np.sqrt(generation) 
                    step = levy_flight(num_dimensions)
                    xk = xi + alpha * step
                
                xk = np.clip(xk, bounds[:, 0], bounds[:, 1])
            
            obj_values = zdt1(xk)
            l = np.random.choice(num_nests)
            objective_l = sorted_objectives[l]
            
            if dominates_pair(obj_values, objective_l):
                sorted_nests[l] = xk
                sorted_objectives[l] = obj_values
            
            num_evaluations += 1
        
        # Update the population and objective values
        population = sorted_nests
        objective_values = sorted_objectives
        generation += 1
        
    return sorted_nests, sorted_objectives, generation, pareto_optimal, pareto_front, optimisation_archive


In [ ]:
def generate_zdt1_pareto_front():
    """Generate the original Pareto front for ZDT1."""
    x = np.linspace(0, 1, 2000)
    f1 = x
    f2 = 1 - np.sqrt(f1)
    return f1, f2

In [ ]:
def generate_zdt2_pareto_front():
    """Generate the original Pareto front for ZDT2."""
    x = np.linspace(0, 1, 2000)
    f1 = x
    f2 = 1 - f1**2
    return f1, f2

In [ ]:
def generate_zdt3_pareto_front():
    """Generate the original Pareto front for ZDT3."""
    # Define the discontinuous segments
    segments = [
        (0.0, 0.0830015349),
        (0.1822287280, 0.2577623634),
        (0.4093136748, 0.4538821041),
        (0.6183967944, 0.6525117038),
        (0.8233317983, 0.8518328654),
    ]

    f1 = []
    f2 = []

    for (start, end) in segments:
        x = np.linspace(start, end, 400)  # Increase granularity for each segment
        f1.extend(x)
        f2.extend(1 - np.sqrt(x) - x * np.sin(10 * np.pi * x))

    return np.array(f1), np.array(f2)


In [ ]:
def plot_pareto_front(best_fitness, original_pareto_front, title="Pareto Front Comparison"):
    """Plot the Pareto front for a set of objective values."""
    best_fitness = np.array(best_fitness)
    original_f1, original_f2 = original_pareto_front
    
    plt.figure(figsize=(10, 6))
    plt.scatter(best_fitness[:, 0], best_fitness[:, 1], color='blue', s=50, alpha=0.7, label='MOMCS Solutions')
    plt.plot(original_f1, original_f2, color='red', linewidth=2, label='Original Pareto Front (ZDT1)')
    plt.title(title)
    plt.xlabel('f1')
    plt.ylabel('f2')
    plt.legend()
    plt.ylim(-1,1)
    #plt.xlim(0,1)
    #plt.grid(True)
    plt.show()
    

In [ ]:
"""For ZDT3 Only"""
def plot_pareto_front(best_fitness, original_pareto_front, title="Pareto Front Comparison"):
    """Plot the Pareto front for a set of objective values."""
    best_fitness = np.array(best_fitness)
    original_f1, original_f2 = original_pareto_front
    
    plt.figure(figsize=(10, 6))
    plt.scatter(best_fitness[:, 0], best_fitness[:, 1], color='blue', s=50, alpha=0.7, label='MOMCS Solutions')
    
    # Plot each segment separately
    segments = [
        (0.0, 0.0830015349),
        (0.1822287280, 0.2577623634),
        (0.4093136748, 0.4538821041),
        (0.6183967944, 0.6525117038),
        (0.8233317983, 0.8518328654),
    ]
    
    for start, end in segments:
        x = np.linspace(start, end, 400)
        y = 1 - np.sqrt(x) - x * np.sin(10 * np.pi * x)
        plt.plot(x, y, color='red', linewidth=2)
    
    plt.title(title)
    plt.xlabel('f1')
    plt.ylabel('f2')
    plt.legend()
    plt.ylim(-1,1)
    #plt.xlim(0,1)
    #plt.grid(True)
    plt.show()


In [ ]:
start_total = time.time()

num_nests = 50
max_evaluations = 2500000
A = 1
phi = (1 + np.sqrt(5)) / 2
bounds = np.array([[0, 1]] * 30)  # ZDT1 has 30 dimensions

best_nests, best_fitness, generation, pareto_optimal, pareto_front, optimisation_archive = run_momcs(num_nests, max_evaluations, A, phi, bounds)
print(generation)

# Total execution time
end_total = time.time()
elapsed_total = end_total - start_total
print(f'Total time taken for optimisation: {elapsed_total:.2f} seconds')


In [ ]:
def efficient_non_dominated_sort_first_front(objective_values):
    num_individuals = len(objective_values)
    dominated_count = np.zeros(num_individuals, dtype=int)
    dominates = [set() for _ in range(num_individuals)]

    # Loop to find all dominances
    for i in range(num_individuals):
        for j in range(i + 1, num_individuals):
            if dominates_pair(objective_values[i], objective_values[j]):
                dominates[i].add(j)
                dominated_count[j] += 1
            elif dominates_pair(objective_values[j], objective_values[i]):
                dominates[j].add(i)
                dominated_count[i] += 1

    # Identify the first front
    first_front = np.where(dominated_count == 0)[0].tolist()
    
    # Return only the first front
    return [first_front]


In [ ]:
best_fitness_front = []
pareto_front_front = []
# Non-dominated sorting and crowding distance calculation
fronts = efficient_non_dominated_sort_first_front(best_fitness)
best_fitness_front.extend(np.array(best_fitness[i]) for i in fronts[0])


In [ ]:
# Generate the original Pareto front
original_pareto_front = generate_zdt1_pareto_front()

# Plot the results
plot_pareto_front(pareto_front, original_pareto_front)


In [ ]:
# Generate the original Pareto front
original_pareto_front = generate_zdt1_pareto_front()

# Plot the results
plot_pareto_front(best_fitness, original_pareto_front)


In [ ]:
# Generate the original Pareto front
original_pareto_front = generate_zdt1_pareto_front()

# Plot the results
plot_pareto_front(best_fitness_front, original_pareto_front)


In [ ]:
# Extract all objective values
all_objective_values = []
for _, objectives in optimisation_archive:
    all_objective_values.extend(objectives)

# Convert to numpy array for convenience
all_objective_values = np.array(all_objective_values)

# Separate objectives into x and y
objective1 = all_objective_values[:, 0]
objective2 = all_objective_values[:, 1]

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(objective1, objective2, c='b', marker='o', alpha=0.4)
plt.xlabel('Objective 1')
plt.ylabel('Objective 2')
plt.title('Objective 1 vs Objective 2')
plt.grid(True)
plt.show()


In [ ]:
true_front = np.array(original_pareto_front).T  # shape (2000, 2)
momcs_front = np.array(best_fitness_front)  # shape (num_solutions, 2)
ref_point = [1.1, 1.1]  # Reference point slightly larger than the max values in objective space

from deap.tools._hypervolume import hv

def calculate_hv(front, ref_point):
    """Calculate the hypervolume of a front given a reference point."""
    return hv.hypervolume(front, ref_point)

# Define reference point
ref_point = [1.1, 1.1]  # Slightly larger than the maximum expected values in the objectives

# Calculate HV for the true front and the MOMCS front
true_hv = calculate_hv(true_front, ref_point)
momcs_hv = calculate_hv(momcs_front, ref_point)

print(f"Hypervolume of True Pareto Front: {true_hv}")
print(f"Hypervolume of MOMCS Pareto Front: {momcs_hv}")

In [ ]:
def generational_distance(obtained_front, true_front):
    distances = []
    for point in obtained_front:
        min_dist = np.min(np.linalg.norm(true_front - point, axis=1))
        distances.append(min_dist)
    gd = np.sqrt(np.mean(np.square(distances)))
    return gd

gd = generational_distance(momcs_front, true_front)
print(f"Generational Distance (GD): {gd}")